# Audio to Text

利用Whisper 將語音轉文字
https://huggingface.co/openai/whisper-large-v3

GitHub: https://github.com/openai/whisper




In [2]:
## Whisper Speech Recognition Example

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# result = pipe(sample)
# print(result["text"])

config.json: 100%|██████████| 1.27k/1.27k [00:00<00:00, 1.57MB/s]
model.safetensors: 100%|██████████| 3.09G/3.09G [00:49<00:00, 61.9MB/s]
generation_config.json: 100%|██████████| 3.94k/3.94k [00:00<00:00, 14.0MB/s]
preprocessor_config.json: 100%|██████████| 340/340 [00:00<00:00, 661kB/s]
tokenizer_config.json: 100%|██████████| 283k/283k [00:00<00:00, 531kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.44MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 4.58MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 4.45MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 311kB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 33.6MB/s]
special_tokens_map.json: 100%|██████████| 2.07k/2.07k [00:00<00:00, 10.6MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
## Speech to text
import gradio as gr

title = "語音 to 文字"

def generateAudio(input):
    print(input)
    result = pipe(input)
    return result["text"]

audio_input = gr.Audio(type="filepath")

app = gr.Interface(fn=generateAudio, inputs=audio_input , outputs="text", title=title)
app.launch()

Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


/private/var/folders/b5/tv_lh5616bgcqzvxbrkkkwdw0000gn/T/gradio/53d790f8297ca576d57ba7826d62604c9933ba8a/audio.wav
